# Reply Tool AB Test Post-Deployment QA

[Task](https://phabricator.wikimedia.org/T273096)

Instrumentation changes were deployed to make it possible to identify any non-discussion tool EditAttemptStep events by users in the AB test. The purpose of this post-deployment QA is to confirm that events are logging as expected and needed to run the AB test. 

Per this [gerrit change](https://gerrit.wikimedia.org/r/c/mediawiki/extensions/VisualEditor/+/660076/2/modules/ve-mw/init/ve.init.mw.trackSubscriber.js) it looks like they will be logged as "discussiontools-abtest". Will it be possible to differentiate the two groups?

In [ ]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(magrittr); library(zeallot); library(glue); library(tidyverse); library(zoo); library(lubridate)
    library(scales)
})

In [12]:
# collect all init events by bucket and event type
query <-
"
SELECT
  event.bucket AS test_group,
  wiki As wiki,
  IF(event.integration = 'discussiontools', 'discussiontool', 'non_discussion_tool') AS event_type,
  COUNT (DISTINCT event.user_id) AS user,
  COUNT(*) AS init_events
FROM event.editattemptstep
WHERE
  year = 2021 
-- review wikis where AB test deployed
  AND wiki = 'idwiki'
  AND event.action = 'init'
  AND NOT event.init_type = 'section'
-- remove logged out users that are not in test
  AND event.user_id != 0
GROUP BY 
  event.bucket,
  wiki,
  IF(event.integration = 'discussiontools', 'discussiontool', 'non_discussion_tool') 
"


In [13]:
collect_reply_init_events <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [15]:
collect_reply_init_events

test_group,wiki,event_type,user,init_events
<chr>,<chr>,<chr>,<int>,<int>
test,idwiki,discussiontool,6,6
NULL,idwiki,discussiontool,6,18
NULL,idwiki,non_discussion_tool,994,4003


In [ ]:
# run check to make sure logged out users are not included
